In [2]:
from diophila import OpenAlex
import numpy as np
import pandas as pd
import itertools
import tqdm
import pandas_gbq
from scipy import stats
import matplotlib.pyplot as plt


openalex = OpenAlex()

In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account

#input google cloud credentials
credentials = service_account.Credentials.from_service_account_file("/Users/jeffreyzhou/Desktop/Moore Lab/key.json")
client = bigquery.Client(credentials = credentials, project = "openalex-bigquery")

In [5]:
pandas_gbq.context.credentials = credentials
authors = pd.read_gbq("SELECT * FROM `openalex-bigquery.filtered_author_joined.post-2012-30-10yrs` ", project_id="openalex-bigquery")
authors['TC'] = authors['TC'].astype(int)

In [7]:
def create_df():
    processed_authors = pd.DataFrame()
    years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
    processed_authors['ID'] = pd.NA
    processed_authors['Name'] = pd.NA
    processed_authors['Concept'] = pd.NA
    processed_authors = processed_authors.reindex(processed_authors.columns.to_list() + years, axis=1)
    
    return processed_authors

In [8]:
authors.columns

Index(['id', 'name', 'name_alternative', 'orcid', 'works_count', 'TC',
       'affiliation_name', 'affiliation_id', 'affiliation_ror',
       'affiliation_country', 'affiliation_type', 'works_api_url',
       'total_cite', 'concept_0', 'score_0', 'concept_1', 'score_1',
       'concept_2', 'score_2', 'year_0', 'cited_0', 'year_1', 'cited_1',
       'year_2', 'cited_2', 'year_3', 'cited_3', 'year_4', 'cited_4', 'year_5',
       'cited_5', 'year_6', 'cited_6', 'year_7', 'cited_7', 'year_8',
       'cited_8', 'year_9', 'cited_9', 'year_10', 'cited_10'],
      dtype='object')

In [11]:
# partitions dataset "authors" into concepts, removing extraneous columns and adding missing annual information
# takes about 30 mins to run

all_concepts = {}
years = {'2012':0, '2013':0, '2014':0, '2015':0, '2016':0, '2017':0, '2018':0, '2019':0, '2020':0, '2021':0, '2022':0}

for index, row in tqdm.tqdm(authors.iterrows()):
    year = years.copy()
    concept = row['concept_0']
    if concept not in all_concepts.keys():
        all_concepts[concept] = create_df()
    
    output = [row['id'], row['name'], concept]
    row_year = row[["year_0", "year_1", "year_2", "year_3", "year_4", "year_5", "year_6", "year_7", "year_8", "year_9", "year_10"]].tolist()
    row_cited = row[["cited_0","cited_1", "cited_2", "cited_3", "cited_4", "cited_5", "cited_6", "cited_7", "cited_8", "cited_9", "cited_10"]].tolist()
    
    for i in range(len(row_year)):
        if row_year[i]:
            year[row_year[i]] = row_cited[i]
    
    x, y = zip(*year.items())
    output = output + list(y)
    
    
    all_concepts[concept].loc[len(all_concepts[concept].index)] = output
    

587366it [31:20, 312.31it/s]


In [23]:
concepts = all_concepts.keys()

for c in concepts:
    all_concepts[c]['First_5'] = all_concepts[c]['2012'] + all_concepts[c]['2013'] + all_concepts[c]['2014'] + all_concepts[c]['2015'] + all_concepts[c]['2016']
    all_concepts[c]['Next_6'] = all_concepts[c]['2017'] + all_concepts[c]['2018'] + all_concepts[c]['2019'] + all_concepts[c]['2020'] + all_concepts[c]['2021'] + all_concepts[c]['2022']

In [13]:
# Dict of percentiles for all authors in all concepts

c_percent = {c:0 for c in concepts}
for c in concepts:
    f5 = []
    l6 = []
    first = all_concepts[c]['First_5']
    last = all_concepts[c]['Next_6']
    for index, row in tqdm.tqdm(all_concepts[c].iterrows()):
        f5.append(stats.percentileofscore(first, row['First_5'], kind='weak'))
        l6.append(stats.percentileofscore(last, row['Next_6'], kind='weak'))
        
    c_percent[c] = np.array([f5, l6])

28686it [00:01, 16020.25it/s]
16862it [00:00, 21223.97it/s]
3320it [00:00, 25285.56it/s]
5180it [00:00, 25461.64it/s]
52955it [00:03, 14794.63it/s]
48065it [00:03, 15543.87it/s]
5192it [00:00, 24995.90it/s]
14158it [00:00, 21510.61it/s]
91311it [00:08, 10994.38it/s]
163151it [00:21, 7658.78it/s]
57953it [00:04, 13977.20it/s]
4074it [00:00, 25288.62it/s]
3534it [00:00, 25505.58it/s]
4886it [00:00, 25258.36it/s]
55353it [00:03, 14612.34it/s]
15381it [00:00, 21782.17it/s]
5177it [00:00, 25273.51it/s]
1303it [00:00, 25830.56it/s]
10825it [00:00, 23362.89it/s]


In [14]:
#function that plots graphs of specific width and height.

def plot_func(x, y, xlabel, ylabel, title):
    fig = plt.figure(figsize=(16, 8))
    a, b = np.polyfit(x, y, deg=1)
    plt.scatter(x, y, alpha=1200/len(x), linewidths= 1)
    plt.plot(x, a*x+b)
    fig.suptitle(title, fontsize=24)
    plt.xlabel(xlabel, fontsize=18)
    plt.ylabel(ylabel, fontsize=16)

    plt.show()

In [ ]:
for c in concepts:
    plot_func(c_percent[c][0], c_percent[c][1], 'Percentile in First 5 years', 'Percentile in Last 6 years', c)

In [143]:
allcited = authors[["cites_0","cited_1", "cited_2", "cited_3", "cited_4", "cited_5", "cited_6", "cited_7", "cited_8", "cited_9", "cited_10"]]

In [167]:
allyears = authors[["year_0", "year_1", "year_2", "year_3", "year_4", "year_5", "year_6", "year_7", "year_8", "year_9", "year_10"]]

In [ ]:
# Run this cell to print and export the data in dataframe into a spreadsheet

for c in concepts:
    name_of_file = c # Change this string to desired filename
    all_concepts[c].to_excel(f'{name_of_file}.xlsx')

In [117]:
concepts

dict_keys(['Political science', 'Philosophy', 'Economics', 'Business', 'Psychology', 'Mathematics', 'Geology', 'Environmental science', 'Biology', 'Medicine', 'Computer science', 'Geography', 'History', 'Materials science', 'Physics', 'Chemistry', 'Art', 'Sociology', 'Engineering'])

In [137]:
# Lagged regression
import statsmodels.api as sm

c = 'Engineering'

curr = all_concepts[c].drop(['First_5', 'Next_6', 'ID', 'Name', 'Concept'], axis=1)
curr = curr.transpose()

l = []
for i in range(len(curr.T)):
    lag = curr[i].shift()
    l.append(lag)

lagged = pd.DataFrame(l).transpose()

model = sm.OLS(curr.melt()['value'],lagged.melt()['value'], missing='drop')
result = model.fit()

print(c)
print(result.summary())

Engineering
                                 OLS Regression Results                                
Dep. Variable:                  value   R-squared (uncentered):                   0.810
Model:                            OLS   Adj. R-squared (uncentered):              0.810
Method:                 Least Squares   F-statistic:                          4.603e+05
Date:                Wed, 22 Feb 2023   Prob (F-statistic):                        0.00
Time:                        13:39:06   Log-Likelihood:                     -5.1225e+05
No. Observations:              108250   AIC:                                  1.025e+06
Df Residuals:                  108249   BIC:                                  1.025e+06
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------

In [116]:
result.tvalues

value    678.451158
dtype: float64

In [93]:
art = all_concepts['Economics'].drop(['First_5', 'Next_6', 'ID', 'Name', 'Concept'], axis=1)

In [97]:
import statsmodels.api as sm
model = sm.OLS(art.melt()['value'],lagged.melt()['value'], missing='drop')
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  value   R-squared (uncentered):                   0.799
Model:                            OLS   Adj. R-squared (uncentered):              0.799
Method:                 Least Squares   F-statistic:                          1.322e+05
Date:                Wed, 22 Feb 2023   Prob (F-statistic):                        0.00
Time:                        13:12:12   Log-Likelihood:                     -1.6009e+05
No. Observations:               33200   AIC:                                  3.202e+05
Df Residuals:                   33199   BIC:                                  3.202e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
value          0.9580      0.003    363.583      0.000       0.953       0.963
==============================================================================
Omnibus:                    77715.881   Durbin-Watson:                   1.737
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       4196133283.999
Skew:                          22.528   Prob(JB):                         0.00
Kurtosis:                    1744.068   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""